In [1]:
import sys
print(sys.executable)




C:\Users\Johanna\anaconda3\envs\AI_project\python.exe


In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score


In [4]:
df_lagged_all = pd.read_parquet("data/processed/df_lagged_all.parquet")
df_2_full = pd.read_parquet("data/processed/df_2_full_v2.parquet")




In [5]:
target_sensor = 1076
same_portal="55620"
neighbour_portal = "56160"

same_portal_sensors = df_2_full[df_2_full['PORTAL_clean'] == same_portal]['DP_ID'].unique()
same_sensors = [s for s in same_portal_sensors if s != target_sensor]


neighbour_sensors = df_2_full[df_2_full['PORTAL_clean'] == neighbour_portal]['DP_ID'].unique()
all_sensors=df_2_full['DP_ID'].unique()
except_target_sensors = [s for s in all_sensors if s != target_sensor]




In [6]:
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(15)
]

In [7]:
def create_sequences(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i+seq_length:i+seq_length+horizon])
    return np.array(Xs), np.array(ys)


In [10]:


X = df_lagged_all[flow_features_neighbour].values
y = df_lagged_all['FLOW_filled'].values


X_seq, y_seq = create_sequences(X, y, seq_length=15, horizon=15)


In [11]:
n_features = X_seq.shape[2]

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features)),
    Dense(15)  # 15 Output-Werte für 15-Minuten-Vorhersage
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)


y_pred = model.predict(X_test)




Epoch 1/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1822 - mae: 3.0382 - val_loss: 19.1656 - val_mae: 3.2000
Epoch 2/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1800 - mae: 3.0370 - val_loss: 19.5491 - val_mae: 3.2230
Epoch 3/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1676 - mae: 3.0373 - val_loss: 19.3715 - val_mae: 3.2295
Epoch 4/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1596 - mae: 3.0352 - val_loss: 19.4853 - val_mae: 3.2318
Epoch 5/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 16.1458 - mae: 3.0351 - val_loss: 19.3683 - val_mae: 3.2075
Epoch 6/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1282 - mae: 3.0324 - val_loss: 19.3993 - val_mae: 3.2288
Epoch 7/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 16.1214 - mae: 3.0308 - val_loss: 19.4030 - val_mae: 3.2125
Epoch 8/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - loss: 16.1265 - mae: 3.0323 - val_loss: 19.5075 - val_mae: 3.2273
Epoch 9/20
1642/

NameError: name 'mean_squared_error' is not defined

In [21]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LSTM (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_test_sum = y_test.sum(axis=1)
y_pred_sum = y_pred.sum(axis=1)

# Metriken berechnen
rmse = mean_squared_error(y_test_sum, y_pred_sum, squared=False)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)

print(f"LSTM Sum Forecast (FLOW) → RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

LSTM (FLOW) -> RMSE: 4.012, MAE: 2.941, R2: 0.630
LSTM Sum Forecast (FLOW) → RMSE: 37.275, MAE: 23.493, R²: 0.817


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
